This notebook is used as a dev space to test and build python definations

In [166]:
import pandas as pd
import json
import collections, functools, operator
import datetime

In [242]:
test = pd.DataFrame(json.loads('''[{"id": 1, "payer": "DANNON", "points": 2000, "timestamp": "2020-11-02T14:00:00Z" },
             {"id": 2, "payer": "UNILEVER", "points": 200, "timestamp": "2020-10-31T11:00:00Z" },
             {"id": 3, "payer": "DANNON", "points": -200, "timestamp": "2020-10-31T15:00:00Z" },
             {"id": 4, "payer": "MILLER COORS", "points": 100, "timestamp": "2020-11-01T14:00:00Z"},
             {"id": 5, "payer": "DANNON", "points": 300, "timestamp": "2020-10-31T10:00:00Z" }]'''))
test

,id,payer,points,timestamp
0,1,DANNON,2000,2020-11-02T14:00:00Z
1,2,UNILEVER,200,2020-10-31T11:00:00Z
2,3,DANNON,-200,2020-10-31T15:00:00Z
3,4,MILLER COORS,100,2020-11-01T14:00:00Z
4,5,DANNON,300,2020-10-31T10:00:00Z


In [243]:
test['timestamp'] = test['timestamp'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ"))
test['points'] = test.points.astype(int)                                       

In [244]:
test = test.sort_values(by='timestamp')

In [246]:
points = 2000

total = 0
for row in range(len(test)):
#     print(test.iloc[row].points)
    total += test.iloc[row].points

total
if total < points:
    print('You dont have that many points to spend. Total points available are: ', total)
else:
    record_change = []
    for row in range(len(test)):
        if points == 0:
            break
        else:
            #if the number of points is greater than the number availabe 
            #at a payer, zero it out and minus from total points needed to use
            print('points value: ',test.iloc[row].points)
            if points > test.iloc[row].points and test.iloc[row].points >=0:
                print('removing points')
                points -= test.iloc[row].points
                record_change.append({'payer': test.iloc[row].payer, 'points': -test.iloc[row].points})
            elif test.iloc[row].points > points:
                print('removing points')
                record_change.append({'payer': test.iloc[row].payer, 'points': -points})
                points = 0
            

points value:  300
removing points
points value:  200
removing points
points value:  -200
points value:  100
removing points
points value:  2000
removing points


In [247]:
record_change

[{'payer': 'DANNON', 'points': -300},
 {'payer': 'UNILEVER', 'points': -200},
 {'payer': 'MILLER COORS', 'points': -100},
 {'payer': 'DANNON', 'points': -1400}]

In [248]:
pd.DataFrame(record_change).groupby(['payer']).sum().reset_index().set_index('payer').T.to_json(orient="records")

'[{"DANNON":-1700,"MILLER COORS":-100,"UNILEVER":-200}]'

In [ ]:
curl -X POST http://127.0.0.1:8000/transactions/add -H 'Content-Type: application/json' -d '{"payer": "DANNON", "points": 2000, "timestamp": "2020-11-02T14:00:00Z" }'
curl -X POST http://127.0.0.1:8000/transactions/add -H 'Content-Type: application/json' -d '{"payer": "UNILEVER", "points": 200, "timestamp": "2020-10-31T11:00:00Z" }'
curl -X POST http://127.0.0.1:8000/transactions/add -H 'Content-Type: application/json' -d '{"payer": "DANNON", "points": -200, "timestamp": "2020-10-31T15:00:00Z" }'
curl -X POST http://127.0.0.1:8000/transactions/add -H 'Content-Type: application/json' -d '{"payer": "MILLER COORS", "points": 100, "timestamp": "2020-11-01T14:00:00Z"}'
curl -X POST http://127.0.0.1:8000/transactions/add -H 'Content-Type: application/json' -d '{"payer": "DANNON", "points": 300, "timestamp": "2020-10-31T10:00:00Z" }'